In [3]:
import pandas as pd
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [4]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("StopWords").getOrCreate()

In [5]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

In [6]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep=r"\t", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [7]:
#Number of rows 
df.count()

1785997

In [ ]:
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'string'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'string'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'string'),
 ('total_votes', 'string'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [8]:

# Cast columns customer_id, product_id, star_rating, helpful_votes, total_votes to integer type
df = df.withColumn("customer_id", 
                                  df["customer_id"]
                                  .cast('int'))

df = df.withColumn("product_parent", 
                                  df["product_parent"]
                                  .cast('int'))

df = df.withColumn("star_rating", 
                                  df["star_rating"]
                                  .cast('int'))

df = df.withColumn("helpful_votes", 
                                  df["helpful_votes"]
                                  .cast('int'))

df = df.withColumn("total_votes", 
                                  df["total_votes"]
                                  .cast('int'))

df = df.withColumn("review_date", 
                                  df["review_date"]
                                  .cast('date'))
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [10]:
products_df = df.select(["product_id", "product_title"]).distinct()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CJ7IUI6|The Elder Scrolls...|
|B00DHF39KS|Wolfenstein: The ...|
|B00MUTAVH6|Under Night In-Bi...|
|B001AZSEUW|              Peggle|
|B00KVOVBGM|PlayStation 4 Con...|
|B00O9VGH4Y|USPRO&reg; Headph...|
|B004OQNZY4|Phineas and Ferb:...|
|B00ZLN980O|Donop seablue 2.4...|
|B002L8W5V6|Dotop Nintendo Ga...|
|B007AJZ5PY|Nyko Game Case fo...|
|B000AOEU2K|Fire Emblem: Path...|
|B000H8BW7U|Tanarus (PC) (Com...|
|B013RADQOQ|Susenstone® 2400D...|
|B00KQXKUJ2|FIFA 15 (Ultimate...|
|B006W41X2C|Turtle Beach - Ea...|
|B000KCX9M4|Grand Theft Auto:...|
|B00YT90JWC|Red Wii Mini Cons...|
|B0096KG6A8|Wii U Super Mario...|
|B00L6AVLB0|World of Tanks-X3...|
|B000IMYKQ0|Wii Nunchuk Contr...|
+----------+--------------------+
only showing top 20 rows



In [9]:
review_id_df = df.select(["review_id", "product_id", "product_parent", "review_date"])
review_id_df.show()

+--------------+----------+--------------+-----------+
|     review_id|product_id|product_parent|review_date|
+--------------+----------+--------------+-----------+
| RTIS3L2M1F5SM|B001CXYMFS|     737716809| 2015-08-31|
| R1ZV7R40OLHKD|B00M920ND6|     569686175| 2015-08-31|
|R3BH071QLH8QMC|B0029CSOD2|      98937668| 2015-08-31|
|R127K9NTSXA2YH|B00GOOSV98|      23143350| 2015-08-31|
|R32ZWUXDJPW27Q|B00Y074JOM|     821342511| 2015-08-31|
|R3AQQ4YUKJWBA6|B002UBI6W6|     328764615| 2015-08-31|
|R2F0POU5K6F73F|B008XHCLFO|      24234603| 2015-08-31|
|R3VNR804HYSMR6|B00BRA9R6A|     682267517| 2015-08-31|
| R3GZTM72WA2QH|B009EPWJLA|     435241890| 2015-08-31|
| RNQOY62705W1K|B0000AV7GB|     256572651| 2015-08-31|
|R1VTIA3JTYBY02|B00008KTNN|     384411423| 2015-08-31|
|R29DOU8791QZL8|B000A3IA0Y|     472622859| 2015-08-31|
|R15DUT1VIJ9RJZ|B0053BQN34|     577628462| 2015-08-31|
|R3IMF2MQ3OU9ZM|B002I0HIMI|     988218515| 2015-08-31|
|R23H79DHOZTYAU|B0081EH12M|     770100932| 2015-08-31|
| RIV24EQA

In [11]:
vine_df = df.select(["review_id", "star_rating","helpful_votes","total_votes"])
vine_df.show()

+--------------+-----------+-------------+-----------+
|     review_id|star_rating|helpful_votes|total_votes|
+--------------+-----------+-------------+-----------+
| RTIS3L2M1F5SM|          5|            0|          0|
| R1ZV7R40OLHKD|          5|            0|          0|
|R3BH071QLH8QMC|          1|            0|          1|
|R127K9NTSXA2YH|          3|            0|          0|
|R32ZWUXDJPW27Q|          4|            0|          0|
|R3AQQ4YUKJWBA6|          1|            0|          0|
|R2F0POU5K6F73F|          5|            0|          0|
|R3VNR804HYSMR6|          5|            0|          0|
| R3GZTM72WA2QH|          5|            0|          0|
| RNQOY62705W1K|          4|            0|          0|
|R1VTIA3JTYBY02|          5|            0|          0|
|R29DOU8791QZL8|          1|            0|          0|
|R15DUT1VIJ9RJZ|          2|            0|          0|
|R3IMF2MQ3OU9ZM|          4|            0|          0|
|R23H79DHOZTYAU|          1|            1|          1|
| RIV24EQA

In [12]:
customers_df = df.groupBy("customer_id").count()
customers_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   48670265|    1|
|   49103216|    2|
|    1131200|    1|
|   43076447|    2|
|   46261368|    1|
|    4883305|    5|
|   41192649|    1|
|   40985731|    7|
|   10437900|    2|
|   22245671|    1|
|    2574873|    1|
|    4696154|    1|
|    5621202|    1|
|    5871933|    2|
|   44089812|    1|
|    2845910|    1|
|    5274369|    1|
|   39069693|    2|
|     137793|    1|
|   31914942|    3|
+-----------+-----+
only showing top 20 rows



In [13]:
customers_df = customers_df.withColumnRenamed("count","customer_count")

customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [14]:
customers_df = customers_df.withColumn("customer_count", 
                                  customers_df["customer_count"]
                                  .cast('int'))

customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   48670265|             1|
|   49103216|             2|
|    1131200|             1|
|   43076447|             2|
|   46261368|             1|
|    4883305|             5|
|   41192649|             1|
|   40985731|             7|
|   10437900|             2|
|   22245671|             1|
|    2574873|             1|
|    4696154|             1|
|    5621202|             1|
|    5871933|             2|
|   44089812|             1|
|    2845910|             1|
|    5274369|             1|
|   39069693|             2|
|     137793|             1|
|   31914942|             3|
+-----------+--------------+
only showing top 20 rows



In [15]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.c0qbpuena9xq.us-east-1.rds.amazonaws.com:5432/mypostgresdb"
config = {"user":"root", 
          "password": "Leila0521!",
          "driver":"org.postgresql.Driver"}

In [16]:
# Write DataFrame to review_id table in RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [17]:
# Write DataFrame to review_id table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

Py4JJavaError: ignored

In [18]:
# Write DataFrame to review_id table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

Py4JJavaError: ignored

In [19]:
# Write DataFrame to review_id table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

Py4JJavaError: ignored